# Documentation


In this notebook, we aim to evaluate the performance of several pre-fine-tuned YOLO models on a dataset that closely resembles our own. The fine-tuned YOLO models can be accessed from the following GitHub repository: https://github.com/keremberke/awesome-yolov8-models.

For our specific tasks, we have selected three models from the repository:

* keremberke/yolov8n-table-extraction
* keremberke/yolov8s-table-extraction
* keremberke/yolov8m-table-extraction

These models have been fine-tuned on a dataset available at https://huggingface.co/datasets/keremberke/table-extraction?row=0.

Throughout the notebook, we will demonstrate how to feed our test dataset into these YOLO models, keeping in mind the specific input requirements of the YOLO architecture.

#  Installing Libraries

In [ ]:
! pip install ultralyticsplus==0.0.23 ultralytics==8.0.21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=7119d91ee47616a770a5089d504000ffe915159a18e893849deb555cfe42c698
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76

# Setting the Working Directory

In [ ]:
import os

# set the working directory (this version is created for Google Colab)
my_wd = "/content"
os.chdir(my_wd)
print("Changed working directory to:", os.getcwd())

Changed working directory to: /content


# Mounting Google Drive

In [ ]:
from google.colab import drive

# mount google drive
drive.mount(os.path.join(my_wd,"drive"))

Mounted at /content/drive


# Loading and Unzipping the Folder Conatining our Dataset to our Working Directory

In [ ]:
# define the google drive location
drive_location = "drive/your_path.zip"

# create the final path
zip_file_path = os.path.join(my_wd, drive_location)

# keep the name of the zipped folder
zip_basename = os.path.splitext(os.path.basename(drive_location))[0]

# unzip the folder
!unzip -q "{zip_file_path}" -d "{zip_basename}"

# Data Organization to Train, Val, Test Sets

This section is pivotal as it involves two critical tasks:

1. Creating our training, testing, and validation sets.
1. Generating a YAML file containing essential data information required for the YOLO model to comprehend the dataset structure and annotations.


Due to the strict file-reading structure of YOLOv8, it is crucial to organize the data in a specific manner. The files need to be grouped into three folders: `train`, `val`, and `test` for both `images` and `labels`. The provided code below accomplishes this task.

First we load the data created in the dataset annotation step, from Google Drive

In [ ]:
# define the google drive location
drive_location = "drive/your_path/Data for YOLOv8.zip"

# create the final path
zip_file_path = os.path.join(my_wd, drive_location)

# keep the name of the zipped folder
zip_basename = os.path.splitext(os.path.basename(drive_location))[0]

# unzip the folder
!unzip -q "{zip_file_path}" -d "{zip_basename}"

 Now, we create our training, testing, and validation sets.

In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil

# Set the paths to your dataset folders
images_folder_path = "your_path/Data for YOLOv8/images"
images_folder_path = os.path.join(my_wd, images_folder_path)

labels_folder_path = "your_path/Data for YOLOv8/labels"
labels_folder_path = os.path.join(my_wd, labels_folder_path)

# Set the paths for the training, validation, and test sets
train_images_path = "your_path/Data for YOLOv8/images/train"
train_images_path = os.path.join(my_wd, train_images_path)

val_images_path = "your_path/Data for YOLOv8/images/val"
val_images_path = os.path.join(my_wd, val_images_path)

test_images_path = "your_path/Data for YOLOv8/images/test"
test_images_path = os.path.join(my_wd, test_images_path)

train_labels_path = "your_path/Data for YOLOv8/labels/train"
train_labels_path = os.path.join(my_wd, train_labels_path)

val_labels_path = "your_path/Data for YOLOv8/labels/val"
val_labels_path = os.path.join(my_wd, val_labels_path)

test_labels_path = "your_path/Data for YOLOv8/labels/test"
test_labels_path = os.path.join(my_wd, test_labels_path)

# Create the test, validation, and training folders if they don't exist
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

# Get the list of image files in the images folder
image_files = [f for f in os.listdir(images_folder_path) if f.endswith(('.jpg', '.png', '.jpeg', ".txt"))]

# Split the dataset into training, validation, and test sets for images
train_files, test_val_files = train_test_split(image_files, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(test_val_files, test_size=0.5, random_state=42)

# Move the training image files to the training folder
for file in train_files:
    src_image_path = os.path.join(images_folder_path, file)
    dest_image_path = os.path.join(train_images_path, file)
    shutil.copy(src_image_path, dest_image_path)

# Move the validation image files to the validation folder
for file in val_files:
    src_image_path = os.path.join(images_folder_path, file)
    dest_image_path = os.path.join(val_images_path, file)
    shutil.copy(src_image_path, dest_image_path)

# Move the test image files to the test folder
for file in test_files:
    src_image_path = os.path.join(images_folder_path, file)
    dest_image_path = os.path.join(test_images_path, file)
    shutil.copy(src_image_path, dest_image_path)

# Move the corresponding label files to the training folder
for file in train_files:
    base_name, _ = os.path.splitext(file)
    label_file = base_name + '.txt'

    src_label_path = os.path.join(labels_folder_path, label_file)
    dest_label_path = os.path.join(train_labels_path, label_file)
    shutil.copy(src_label_path, dest_label_path)

# Move the corresponding label files to the validation folder
for file in val_files:
    base_name, _ = os.path.splitext(file)
    label_file = base_name + '.txt'

    src_label_path = os.path.join(labels_folder_path, label_file)
    dest_label_path = os.path.join(val_labels_path, label_file)
    shutil.copy(src_label_path, dest_label_path)

# Move the corresponding label files to the test folder
for file in test_files:
    base_name, _ = os.path.splitext(file)
    label_file = base_name + '.txt'

    src_label_path = os.path.join(labels_folder_path, label_file)
    dest_label_path = os.path.join(test_labels_path, label_file)
    shutil.copy(src_label_path, dest_label_path)


Now, we are generating the YAML file containing essential data information required for the YOLO model to comprehend the dataset structure and annotations. We specify the path, the train, val and test folders' paths

In [ ]:
import yaml

# Define the information inside yaml
data = {'path': '/your_path/Data for YOLOv8', 'train': 'images/train', 'val': 'images/test', 'test': 'images/val', 'names': {0: 'table'}}

# Define the path to the YAML file
yaml_file_path = 'Data for YOLOv8/yaml_file.yaml'
yaml_file_path = os.path.join(my_wd, yaml_file_path)

# Dump the data to a YAML-formatted string
yaml_content = yaml.dump(data, default_flow_style=False, sort_keys=False)

# Write the YAML content to a file
with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)

print(f'YAML file has been created at: {yaml_file_path}')


YAML file has been created at: /content/Data for YOLOv8/yaml_file.yaml


# Model Evaluation

## yolov8n-table-extraction

Here we evaluate the `keremberke/yolov8n-table-extraction` on our test dataset

In [ ]:
import os
from ultralyticsplus import YOLO, render_result

# Load a model
model = YOLO('keremberke/yolov8n-table-extraction')

# Load a model on our Data
model.val(data=yaml_file_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/6.22M [00:00<?, ?B/s]

Ultralytics YOLOv8.0.21 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs
100%|██████████| 755k/755k [00:00<00:00, 25.7MB/s]
val: Scanning /content/Data for YOLOv8/Data for YOLOv8/labels/test... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<00:00, 749.01it/s]
val: New cache created: /content/Data for YOLOv8/Data for YOLOv8/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.98s/it]
                   all         18         31     0.0152          1      0.333      0.237
              bordered         18         31     0.0152          1      0.333      0.237
Speed: 0.2ms pre-process, 30.7ms inference, 0.0ms loss, 36.6ms post-process per image


## yolov8s-table-extraction

Here we evaluate the `keremberke/yolov8s-table-extraction` on our test dataset

In [ ]:
import os
from ultralyticsplus import YOLO, render_result

# Load a model
model = YOLO('keremberke/yolov8s-table-extraction')

# Load a model on our Data
model.val(data=yaml_file_path)

config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Ultralytics YOLOv8.0.21 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/Data for YOLOv8/Data for YOLOv8/labels/test.cache... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]
                   all         18         31      0.292      0.613      0.353      0.235
              bordered         18         31      0.292      0.613      0.353      0.235
Speed: 0.2ms pre-process, 37.3ms inference, 0.0ms loss, 0.8ms post-process per image


## yolov8m-table-extraction

Here we evaluate the `keremberke/yolov8m-table-extraction` on our test dataset

In [ ]:
import os
from ultralyticsplus import YOLO, render_result

# Load a model
model = YOLO('keremberke/yolov8m-table-extraction')

# Load a model on our Data
model.val(data=yaml_file_path)

config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/52.0M [00:00<?, ?B/s]

Ultralytics YOLOv8.0.21 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/Data for YOLOv8/Data for YOLOv8/labels/test.cache... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]
                   all         18         31      0.212       0.71      0.673      0.528
              bordered         18         31      0.212       0.71      0.673      0.528
Speed: 0.2ms pre-process, 43.6ms inference, 0.0ms loss, 0.8ms post-process per image


# Single Image Inference

After running the code, the files are automatically saved in the `runs` folder created by the process.

In [ ]:
# Run inference on 'bus.jpg' with arguments
model.predict('/content/your_path.png', save=True, imgsz=640, conf=0.75, iou=0.75)

Ultralytics YOLOv8.0.21 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
image 1/1 /content/bonny 18_img_page58.png: 320x256 1 borderless, 60.4ms
Speed: 0.4ms pre-process, 60.4ms inference, 26.4ms postprocess per image at shape (1, 3, 320, 320)
Results saved to runs/detect/predict


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([1, 6])
 dtype: torch.float32
  + tensor([[1.92000e+02, 4.77000e+02, 1.65400e+03, 1.06800e+03, 6.44643e-01, 1.00000e+00]], device='cuda:0')]

# Extracting All Inferences

Now we will extract the inferences in a folder

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# Specify the path for the new folder
folder_path = 'your output folder name'
folder_path = os.path.join(my_wd, folder_path)

# Create the folder
os.makedirs(folder_path, exist_ok=True)

#
input_folder = 'your input folder name'
input_folder = os.path.join(my_wd, input_folder)

output_folder = folder_path

# List all files in the input folder
input_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

# Wrap the input_files list with tqdm to add a progress bar
for input_file in tqdm(input_files, desc="Processing images", unit="image"):
    # Load the image
    image_path = os.path.join(input_folder, input_file)

    # Perform inference
    results = model.predict(image_path, imgsz=640, conf=0.75, iou=0.75)

    for result in results:
        boxes = result.boxes  # Boxes object for bbox outputs

    # Convert the 'Boxes' object to a list
    bounding_boxes = boxes.xyxy.cpu().numpy().tolist()

    # Save results to the output folder
    output_file = os.path.join(output_folder, input_file)
    render = render_result(model=model, image=image_path, result=results[0])
    render.save(output_file)

print("")
print("Inference and saving completed.")

# Zipping the Results

In [ ]:
# Define the path
folder_name = "your folder name"

# Create the final path
zip_folder_path = os.path.join(my_wd, folder_name + ".zip")

# Zip the folder
!zip -r -q "{zip_folder_path}" "{folder_name}"